In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow import keras
from scipy.fft import fft,fftfreq

In [2]:
x_train = pd.read_csv("../Dataset2/Train/Voltage_L1_train.csv")
y_train = pd.read_csv("../Dataset2/Train/output_train.csv")
x_test = pd.read_csv("../Dataset2/Test/Voltage_L1_test.csv")
y_test = pd.read_csv("../Dataset2/Test/output_test.csv")

In [3]:
print("x_train",x_train.shape)
print("y_train",y_train.shape)
print("x_test",x_test.shape)
print("y_test",y_test.shape)

x_train (5999, 256)
y_train (5999, 1)
x_test (3599, 256)
y_test (3599, 1)


In [4]:
header =[]
for i in range(1,x_train.shape[1]+1):
    header.append("Col"+str(i))
  
x_train.columns = header
x_test.columns = header

In [5]:
header = ["output"]
y_train.columns = header
y_test.columns = header

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.20, random_state=42)

In [7]:
y_train_hot = pd.get_dummies(y_train['output'])
y_test_hot = pd.get_dummies(y_test['output'])
y_val_hot = pd.get_dummies(y_val['output'])

In [8]:
y_train = y_train_hot.to_numpy()
y_test = y_test_hot.to_numpy()
y_val = y_val_hot.to_numpy()
print("y_train:",y_train.shape)
print("y_test:",y_test.shape)
print("y_val:",y_val.shape)
no_of_classes = y_train.shape[1]

y_train: (4799, 6)
y_test: (3599, 6)
y_val: (1200, 6)


In [9]:
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
x_val = x_val.to_numpy()
for i in range(0,x_train.shape[0]):
    x_train[i][:] = fft(x_train[i][:])
    
for i in range(0,x_test.shape[0]):
    x_test[i][:] = fft(x_test[i][:])

for i in range(0,x_val.shape[0]):
    x_val[i][:] = fft(x_val[i][:])

<ipython-input-9-8af77d55593a>:5: ComplexWarning: Casting complex values to real discards the imaginary part
  x_train[i][:] = fft(x_train[i][:])
<ipython-input-9-8af77d55593a>:8: ComplexWarning: Casting complex values to real discards the imaginary part
  x_test[i][:] = fft(x_test[i][:])
<ipython-input-9-8af77d55593a>:11: ComplexWarning: Casting complex values to real discards the imaginary part
  x_val[i][:] = fft(x_val[i][:])


In [10]:
transform = StandardScaler()
x_train_tr = transform.fit_transform(x_train)
x_test_tr = transform.fit_transform(x_test)
x_val_tr = transform.fit_transform(x_val)

In [11]:
x_train = x_train.reshape(x_train_tr.shape[0],x_train_tr.shape[1], 1)
x_test = x_test.reshape(x_test_tr.shape[0],x_test_tr.shape[1], 1)
x_val = x_val.reshape(x_val_tr.shape[0],x_val_tr.shape[1], 1)

In [12]:
print("Training",x_train.shape)
print(y_train.shape)
print("Validation",x_val.shape)
print(y_val.shape)
print("Test",x_test.shape)
print(y_test.shape)
sampling_rate = x_train.shape[1]

Training (4799, 256, 1)
(4799, 6)
Validation (1200, 256, 1)
(1200, 6)
Test (3599, 256, 1)
(3599, 6)


In [13]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv1D(
        filters=hp.Int('conv_1_filter', min_value=64, max_value=256),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(256,1)
    ),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(pool_size=(2)),
    keras.layers.Conv1D(
        filters=hp.Int('conv_2_filter', min_value=64, max_value=256),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(pool_size=(2)),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=16, max_value=128),
        activation='relu'
    ),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=16, max_value=128),
        activation='relu'
    ),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=16, max_value=128),
        activation='relu'
    ),
    keras.layers.Dense(6, activation='softmax')
  ])
  
  model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [14]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [16]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='content',project_name="Classification")

In [17]:
tuner_search.search(x_train,y_train,epochs=10,validation_data=(x_val, y_val))

Trial 5 Complete [00h 00m 59s]
val_accuracy: 0.9633333086967468

Best val_accuracy So Far: 0.9633333086967468
Total elapsed time: 00h 09m 46s
INFO:tensorflow:Oracle triggered exit


In [18]:
model=tuner_search.get_best_models(num_models=1)[0]

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 254, 242)          968       
_________________________________________________________________
batch_normalization (BatchNo (None, 254, 242)          968       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 127, 242)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 125, 100)          72700     
_________________________________________________________________
batch_normalization_1 (Batch (None, 125, 100)          400       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 62, 100)           0         
_________________________________________________________________
flatten (Flatten)            (None, 6200)              0

In [20]:
model.fit(x_train, y_train, epochs=30, validation_data=(x_val, y_val), initial_epoch=10)

Epoch 11/30
150/150 [==============================] - 14s 89ms/step - loss: 0.0979 - accuracy: 0.9725 - val_loss: 0.2328 - val_accuracy: 0.9525
Epoch 12/30
150/150 [==============================] - 11s 73ms/step - loss: 0.1127 - accuracy: 0.9685 - val_loss: 0.1608 - val_accuracy: 0.9608
Epoch 13/30
150/150 [==============================] - 13s 89ms/step - loss: 0.1007 - accuracy: 0.9736 - val_loss: 0.1634 - val_accuracy: 0.9600
Epoch 14/30
150/150 [==============================] - 13s 86ms/step - loss: 0.0826 - accuracy: 0.9766 - val_loss: 0.1616 - val_accuracy: 0.9592
Epoch 15/30
150/150 [==============================] - 13s 88ms/step - loss: 0.0635 - accuracy: 0.9821 - val_loss: 0.1741 - val_accuracy: 0.9608
Epoch 16/30
150/150 [==============================] - 13s 88ms/step - loss: 0.0610 - accuracy: 0.9822 - val_loss: 0.1615 - val_accuracy: 0.9600
Epoch 17/30
150/150 [==============================] - 13s 90ms/step - loss: 0.0648 - accuracy: 0.9814 - val_loss: 0.1698 - val_ac

In [21]:
pred_acc = model.evaluate(x_test,y_test)
print("Test accuracy is {}".format(pred_acc))

113/113 [==============================] - 2s 15ms/step - loss: 0.1303 - accuracy: 0.9744
Test accuracy is [0.13032472133636475, 0.9744373559951782]


In [22]:
model.save("CNN_kerastuner_model.h5")